In [2]:
from ultralytics import YOLO
import json
import pandas as pd
from yolo_config import Config
import os

## Agregate Folds

In [4]:
dataset = 'InBreast_Yolov8'

keys = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 
        'metrics/mAP50-95(B)', 'fitness']

metrics_by_model = []
index = []

for model_size in Config.MODEL_SIZES:
    
    metrics_path = os.path.join(
        Config.RUNS_DIR, 
        Config.DATASET, 
        model_size,
        Config.METRICS_FILE
    )
    metrics = pd.read_csv(metrics_path)
    
    metrics_by_model.append(metrics.loc['mean'])
    index.append(model_size)
    

output_path = os.path.join(
    Config.RUNS_DIR,
    Config.DATASET,
    'mean_metrics.csv'
)
metrics_by_model = pd.DataFrame(metrics_by_model, index=index)
metrics_by_model.to_csv(output_path)

{'metrics/precision(B)': 0.0, 'metrics/recall(B)': 0.0, 'metrics/mAP50(B)': 0.0, 'metrics/mAP50-95(B)': 0.0, 'fitness': 0.0, 'CM(0,0)': 0.0, 'CM(0,1)': 0.0, 'CM(0,2)': 0.0, 'CM(1,0)': 0.0, 'CM(1,1)': 0.0, 'CM(1,2)': 0.0, 'CM(2,0)': 0.8, 'CM(2,1)': 0.6, 'CM(2,2)': 0.0}
